In [ ]:
# Script for non-productive use. It takes the list of URLs from a CSV file,
# downloads each file and then uploads it to the GCS and at the end, 
# creates a CSV compatible with the Vision API Product Search .

# mateuswagner@google.com

# Requirements:
#  Cloud SDK: https://cloud.google.com/sdk
#  pip install google-cloud-storage requests pandas

In [ ]:
# Create a destination bucket
BUCKET_NAME = "fb790ef7-2d45-4ba0-b44e-040d2bd3e244" # change it!
!gsutil mb -l us-east1 gs://$BUCKET_NAME

In [ ]:
import pandas as pd
import requests
import io
import os
import requests
from google.cloud import storage
import logging
import time
import random

log = logging.getLogger(__name__)
logging.basicConfig(level=os.environ.get("LOGLEVEL", "INFO"))

In [ ]:
# Load product set csv

MAX_IMAGE_COUNT = 100 #change it!

df = pd.read_csv('product_set.csv',header=None)
columns = ["image-uri","image-id","product-set-id", "product-id", "product-category"] #, "product-display-name", "labels", "bounding-poly"]
df.columns = columns
df = df.drop_duplicates(subset=['image-uri']) # remove duplicates
df = df[:MAX_IMAGE_COUNT] # truncate to max allowed (20K)


In [ ]:
# Get bucket ref

storage_client = storage.Client()
bucket = storage_client.get_bucket(BUCKET_NAME)


In [ ]:
%%time
# Iterate of rows, download and upload objects

final_df = df.copy()

for index, row in df.iterrows():
    
    log.info("Processing #{}: {}".format(index,row['image-uri']))
    
    blob = bucket.blob(
        "{}/{}.{}".format(row["product-set-id"],row['image-id'],"jpg")
    )
    log.info("Downloading: {}".format(row['image-uri']))
    response = requests.get(
        row['image-uri']
    )
    img_f = io.BytesIO(response.content)
    log.info("Uploading: {}".format(blob))
    blob.upload_from_file(img_f,
                      content_type = response.headers['Content-Type'])

    final_df.at[index,'image-uri'] = "gs://{}/{}".format(BUCKET_NAME, blob.name)

    #time.sleep(random.uniform(0, 2.0)) #optional

In [ ]:
# Save final CSV file

## bulk import require that the number of columns in the csv line should be 8
final_df['product-display-name'] = None
final_df['labels'] = None
final_df['bounding-poly'] = None

final_df.to_csv("final_product_set.csv", header=False, index=False)

In [ ]:
# Copy new CSV to GCS
!gsutil cp final_product_set.csv gs://$BUCKET_NAME